In [2]:
import pandas as pd
import os
import json
import numpy as np

In [7]:
trash = pd.read_csv('/Volumes/WD 2TB/Waze data/Full dump/mx_dump_db2.csv', chunksize=100)
i=0
for chunk in trash:
    i+=1
print(i)

KeyboardInterrupt: 

In [6]:
from pandas.io.json import json_normalize

In [9]:
dump_iterator = pd.read_csv('/Volumes/WD 2TB/Waze data/Full dump/mx_dump_db2.csv', chunksize=100)
for chunk in dump_iterator:
    chunk.to_csv('./sample_dump.csv', mode='a', index=False, header=False)
    if os.path.getsize('./sample_dump.csv') > 20000000:
        break
    
        

In [38]:
df = pd.read_csv('./sample_dump.csv', header=None, names=['id', 'datetime', 'data'])[['data']]
df = vectorized_pull_out_jams(df['data'])


In [37]:
def pull_out_jams(row):
    try:
        row_data_as_json = json.loads(row)
        return {'jams':row_data_as_json['jams'], 'startTime':row_data_as_json['startTime'], 'endTime':row_data_as_json['endTime']}
    except (ValueError, KeyError):
        return None
vectorized_pull_out_jams = np.vectorize(pull_out_jams)


In [14]:
%%timeit
df = pd.read_csv('./sample_dump.csv', header=None, names=['id', 'datetime', 'data'])[['data']]
df['data'] = vectorized_pull_out_jams(df['data'])
df = json_normalize(df['data'].tolist())
df = pd.DataFrame({
      col:np.repeat(df[col].values, df['jams'].str.len())
      for col in df.columns.drop('jams')}
    ).assign(**{'jams':np.concatenate(df['jams'].values)})[df.columns]

df = pd.concat([df.drop('jams', axis=1), pd.DataFrame(df['jams'].tolist())[['level', 'line']]], axis=1)

df = pd.DataFrame({
      col:np.repeat(df[col].values, df['line'].str.len())
      for col in df.columns.drop('line')}
    ).assign(**{'line':np.concatenate(df['line'].values)})[df.columns]

df = pd.concat([df.drop('line', axis=1), pd.DataFrame(df['line'].tolist())], axis=1)

8.3 s ± 723 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%%timeit
lst=[]
reader = pd.read_csv('./sample_dump.csv', header=None, names=['id', 'datetime', 'data'], index_col='datetime', chunksize=1)
for fd in reader:
    upper_json = json.loads(fd['data'][0])
    fd = json_normalize(upper_json['jams'], 'line', ['level'])
    fd.loc[:, 'startTimeMillis'] = upper_json['startTimeMillis']
    lst.append(fd)
    


6.77 s ± 83.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
lst[0].head()

,x,y,speed,length,level,delay,type,uuid,startTimeMillis
0,-99.192319,18.919410,3.347222,708,3,158,NONE,1354947857,1501808760000
1,-99.189473,18.918525,3.347222,708,3,158,NONE,1354947857,1501808760000
2,-99.188905,18.918360,3.347222,708,3,158,NONE,1354947857,1501808760000
3,-99.188539,18.918200,3.347222,708,3,158,NONE,1354947857,1501808760000
4,-99.188204,18.917970,3.347222,708,3,158,NONE,1354947857,1501808760000
